In [ ]:
# This program calculates the average radius of gyration and attack rate of an infection with an input p and τ for 800 time steps.

In [ ]:
using CSV
using Distributions
using LsqFit
using Plots
using PrettyTables
using Random
using Statistics
using JLD

using Test

default(fmt = :png)

In [ ]:
function simpler_jump(x, y, a = 1)
    theta = rand(Uniform(0, 2 * π))
    u = rand(Uniform(0, 1))

    r = a/(3 * u)^(1/3)

    xfl = r * sin(theta)
    yfl = r * cos(theta)

    (floor(Int, x + xfl + 1/2),
    floor(Int, y + yfl + 1/2), r)
end

function outbreakwithjumps_varp(maxtime :: Int,
                           maxtau :: Int,
                           nextstepfcn :: Function,
                           probinfected :: Function,
                           p_arr :: Array{Float64, 1},
                           initialinfectors = [[0, 0, 0, 0]],
                           addinfectors = true) :: Tuple{Array{Int, 1},
                                                   Array{Array{Int}, 1},
                                                   Array{Array{Int}, 1},
                                                   Array{Int64, 1},
                                                   Array{Int64, 1},
                                                   Array{Int64, 1},
                                                   Array{Float64, 1},
                                                   Array{Float64, 1},
                                                   Array{Float64, 1}}
    # Tracking the different agents
    x_start = 0
    y_start = 0

    newinfections = Array{Int64, 1}()
    infectors = initialinfectors
    numinfected = [length(infectors)]
    removed = Set([[x_start, y_start]])
    visited = Set([[x_start, y_start]])
    numvisited = zeros(Int, maxtime)
    numsusceptiblesvisited = zeros(Int, maxtime)
    r_inf = Array{Tuple{Int64, Int64}, 1}()
    graphcomponents = Array{Any, 1}()

    radiusofgyr_inf = zeros(Float64, maxtime)
    radiusofgyr_rem = zeros(Float64, maxtime)
    radiusofgyr_all = zeros(Float64, maxtime)

    com_inf_x = fill(Int[], maxtime)
    com_inf_y = fill(Int[], maxtime)
    com_rem_x = fill(Int[], maxtime)
    com_rem_y = fill(Int[], maxtime)
    com_all_x = fill(Int[], maxtime)
    com_all_y = fill(Int[], maxtime)

    for time in 1:maxtime
        ninfectors = length(infectors)
        ninfected = 0
        newinfections_now = 0

        if time != 1
            numvisited[time] = numvisited[time - 1]
            numsusceptiblesvisited[time] = numsusceptiblesvisited[time - 1]
        end

        for i in 1:ninfectors
            infector = infectors[i]
            (infector[1], infector[2]) = simpler_jump(infector[1], infector[2])
            infector[3] += 1

            #radiusofgyr_inf[time] += (infector[1]^2 + infector[2]^2) # radius squared
            #radiusofgyr_all[time] += (infector[1]^2 + infector[2]^2) # radius squared

            push!(com_inf_x[time], infector[1])
            push!(com_inf_y[time], infector[2])

            if ninfectors != 0
                push!(com_all_x[time], infector[1])
                push!(com_all_y[time], infector[2])
            end

            inremoved = infector[1:2] in removed
            alreadyvisited = infector[1:2] in visited

            if !alreadyvisited
                push!(visited, infector[1:2])
                numvisited[time] += 1
            end

            isinfecting = rand() <= probinfected(infector[1], p_arr[1], p_arr[2])

            if !alreadyvisited && !isinfecting
                numsusceptiblesvisited[time] += 1
            end

            if !inremoved && isinfecting && alreadyvisited
                numsusceptiblesvisited[time] -= 1
            end

            if !inremoved && isinfecting
                push!(removed, infector[1:2])
            end

            if !inremoved && isinfecting
                newinfections_now += 1
                if !addinfectors
                    ninfected += 1
                else
                    push!(infectors, [infector[1:2]..., 0, 0])
            end

                infector[4] += 1
            end
        end

        for site in removed
            push!(com_all_x[time], site[1])
            push!(com_all_y[time], site[2])
            push!(com_rem_x[time], site[1])
            push!(com_rem_y[time], site[2])
        end

        com_inf_x[time] = [round(mean(com_inf_x[time]))]
        com_inf_y[time] = [round(mean(com_inf_y[time]))]
        com_rem_x[time] = [round(mean(com_rem_x[time]))]
        com_rem_y[time] = [round(mean(com_rem_y[time]))]
        com_all_x[time] = [round(mean(com_all_x[time]))]
        com_all_y[time] = [round(mean(com_all_y[time]))]

        for site in infectors
            radiusofgyr_inf[time] += (site[1] - com_inf_x[time][1])^2 + (site[2] - com_inf_y[time][1])^2
            if ninfectors != 0
                radiusofgyr_all[time] += (site[1] - com_all_x[time][1])^2 + (site[2] - com_all_y[time][1])^2
            end
        end

        for site in removed
            radiusofgyr_rem[time] += (site[1] - com_rem_x[time][1])^2 + (site[2] - com_rem_y[time][1])^2
            radiusofgyr_all[time] += (site[1] - com_all_x[time][1])^2 + (site[2] - com_all_y[time][1])^2
        end

        if ninfectors != 0
            radiusofgyr_inf[time] = sqrt(radiusofgyr_inf[time]/ninfectors)
        else
            radiusofgyr_inf[time] = radiusofgyr_inf[time-1]
        end
        radiusofgyr_rem[time] = sqrt(radiusofgyr_rem[time]/(length(removed)))
        radiusofgyr_all[time] = sqrt(radiusofgyr_all[time]/(ninfectors+length(removed)))

        push!(newinfections, newinfections_now)

        infectedtoremove = filter(x -> x[3] >= maxtau, infectors)
        for infector in infectedtoremove
            push!(r_inf, (infector[4], time))
        end

        filter!(x -> x[3] < maxtau, infectors)

        if addinfectors
            push!(numinfected, length(infectors))
        else
            push!(numinfected, ninfected)
        end
    end


    removed = collect(removed)
    
    (numinfected, infectors, removed, newinfections, numsusceptiblesvisited, numvisited, radiusofgyr_inf, radiusofgyr_rem, radiusofgyr_all)

end


In [ ]:
# ------------------------------- EDIT INPUTS BELOW ---------------------------
# no max grid size is used


function probinfected(x, p1, p2)
    if x <= 0
        return p1
    else
        return p2
    end
end

maxtime = 800
# p = 0.48
p_arr = [0.01, 0.01] # p in the left region (x <= 0), p in the right region (i think??)
τ = 50

n_iter = 1000

listofradiusofgyr_inf = fill(Float64[], n_iter)
listofradiusofgyr_rem = fill(Float64[], n_iter)
listofradiusofgyr_all = fill(Float64[], n_iter)
listofsuscp = Array{Array{Int, 1}, 1}()
listofvisit = Array{Array{Int, 1}, 1}()
listofpercent = Array{Array{Float64, 1}, 1}()

t1 = time()

@Threads.threads for i in 1:n_iter
    # (infections, infectors, removed, graphs) = outbreakwithjumps(time, τ, simpler_jump, times, p)
    (~, ~, ~, ~, numsuscpvisited, numvisited, radiusofgyr_inf, radiusofgyr_rem, radiusofgyr_all) = outbreakwithjumps_varp(maxtime, τ, simpler_jump, probinfected, p_arr)
    listofradiusofgyr_rem[i] = radiusofgyr_rem
    listofradiusofgyr_inf[i] = radiusofgyr_inf
    listofradiusofgyr_all[i] = radiusofgyr_all
    push!(listofsuscp, numsuscpvisited)
    push!(listofvisit, numvisited)
    push!(listofpercent, numsuscpvisited./numvisited)
    string("Completed iteration ", i, ".")
end

t2 = time()
t_elapsed = t2-t1
string("Elapsed time is ", t_elapsed)

percent = mean(listofpercent)
stddev = std(listofpercent)

radiusofgyr_inf = mean(listofradiusofgyr_inf)
radiusofgyr_inf_err = std(listofradiusofgyr_inf)/sqrt(n_iter)
radiusofgyr_rem = mean(listofradiusofgyr_rem)
radiusofgyr_rem_err = std(listofradiusofgyr_rem)/sqrt(n_iter)
radiusofgyr_all = mean(listofradiusofgyr_all)
radiusofgyr_all_err = std(listofradiusofgyr_all)/sqrt(n_iter)

#=
plot(1:maxtime,
    [radiusofgyr_inf, radiusofgyr_all],
    label = ["r_inf" "r_all"],
    ribbon=[radiusofgyr_inf_err, radiusofgyr_all_err])
=#

exportfilename = "rofgyr_attrate_p0pt01_tau50" # DO NOT include the file format at the end
# -------------------------------------------------------------------

fn = string(exportfilename, ".jld")
save(fn, "listofradiusofgyr_inf", listofradiusofgyr_inf, "listofradiusofgyr_all", listofradiusofgyr_all,
        "radiusofgyr_inf", radiusofgyr_inf, "radiusofgyr_inf_err", radiusofgyr_inf_err,
        "radiusofgyr_all", radiusofgyr_all, "radiusofgyr_all_err", radiusofgyr_all_err,
        "τ", τ, "p_arr", p_arr, "n_iter", n_iter, "maxtime", maxtime,
        "t_elapsed", t_elapsed,
        "percent", percent, "stddev", stddev,
        "listofsuscp", listofsuscp, "listofvisit", listofvisit,
        "listofpercent", listofpercent,
        "radiusofgyr_rem", radiusofgyr_rem, "radiusofgyr_rem_err", radiusofgyr_rem_err)
